In [19]:
from sentence_transformers import SentenceTransformer, util
import torch
import re

# Cargar modelo multilingüe para embeddings
STmodel = SentenceTransformer('sentence-transformers/LaBSE')  

In [20]:
bug_pattern = re.compile(r"error|bug|fix", re.IGNORECASE)
opt_pattern = re.compile(r"optimiz|mejor|rendimiento", re.IGNORECASE)
version_pattern = re.compile(r"\bversion\b|\bversión\b", re.IGNORECASE)

def exact_match_score(query, text):
    """Calcula un puntaje basado en la coincidencia exacta de palabras clave."""
    query_words = set(query.lower().split())
    text_words = set(text.lower().split())
    return len(query_words & text_words) / len(query_words)

In [21]:
def find_relevant_segment(query, segments, exact_weight=2.0, semantic_weight=1.0):
    query_embedding = STmodel.encode(query, convert_to_tensor=True)
    
    # Extraer el texto de cada segmento para comparación
    segment_texts = [" ".join([seg["version"], *seg["features"], *seg["bugfixes"], *seg["optimizations"]]) for seg in segments]
    segment_embeddings = STmodel.encode(segment_texts, convert_to_tensor=True)
    
    # Similitud semántica
    semantic_scores = util.cos_sim(query_embedding, segment_embeddings).squeeze(0)

    # Similitud exacta
    match_scores = [exact_match_score(query, text) for text in segment_texts]

    # Ponderar ambas similitudes
    scores = [(exact_weight * match_scores[i] + semantic_weight * semantic_scores[i].item()) for i in range(len(segments))]
    
    best_match_idx = scores.index(max(scores))

    best_score = max(scores)
    threshold = 0.9 * best_score  
    close_matches = [segments[i]["version"] for i in range(len(scores)) if scores[i] >= threshold and i != best_match_idx]
    return segments[best_match_idx], close_matches


In [ ]:
def generate_answer( context, close_matches):
    """Genera una respuesta con el changelog completo de la versión encontrada."""
    features = "\n- ".join(context["features"]) if context["features"] else "Sin novedades"
    bugfixes = "\n- ".join(context["bugfixes"]) if context["bugfixes"] else "Sin correcciones"
    optimizations = "\n- ".join(context["optimizations"]) if context["optimizations"] else "Sin optimizaciones"

    if close_matches:
        close_versions = ", ".join(close_matches)
    else:
        close_versions = "No hay versiones cercanas"

    return f"""{context['version']}:
    
Características:
- {features}

Correcciones de errores:
- {bugfixes}

Optimizaciones:
- {optimizations}


Versiones cercanas:
- {close_versions}

"""


In [23]:
def preprocess_context(changelog):
    segments = changelog.split('##')[1:]  # Split by '##' and ignore the first empty split
    processed_segments = []

    for segment in segments:
        lines = segment.strip().split('\n')
        version = lines[0].strip()
        features = []
        bugfixes = []
        optimizations = []

        for line in lines[1:]:
            if re.search(bug_pattern, line):
                bugfixes.append(line.strip('- ').strip())
            elif re.search(opt_pattern, line):
                optimizations.append(line.strip('- ').strip())
            else:
                features.append(line.strip('- ').strip())

        processed_segments.append({
            "version": version,
            "features": features,
            "bugfixes": bugfixes,
            "optimizations": optimizations
        })

    return processed_segments

In [24]:
def main_agent(changelog, query):
    """Procesa la consulta y devuelve la mejor coincidencia con información detallada."""
    segments = preprocess_context(changelog)
    relevant_segment, close_matches = find_relevant_segment(query, segments)
    return generate_answer(relevant_segment, close_matches)


In [26]:
changelog = """
## Version 1.0.0
- Versión inicial de la aplicación.
- Implementación de la funcionalidad de inicio de sesión y registro.
- Creación de la base de datos inicial.
- Diseño básico de la interfaz de usuario.

## Version 1.1.0
- Optimización en la renderización de componentes.
- Corrección de errores menores relacionados con la validación de formularios.
- Mejora en el rendimiento de la carga inicial de la aplicación.
- Ajustes en el diseño para dispositivos móviles.

## Version 1.2.0
- Añadida compatibilidad con el protocolo XMPP.
- Resuelto un problema con el envío de mensajes duplicados.
- Mejora en las notificaciones en tiempo real.
- Traducción de la interfaz a español y francés.

## Version 1.3.0
- Implementación de la funcionalidad para compartir archivos en el chat.
- Añadido soporte para múltiples sesiones por usuario.
- Solucionado un error que impedía la reconexión automática tras la pérdida de conexión.
- Actualización del diseño de la página de perfil.

## Version 1.4.0
- Integración con servicios de terceros para la autenticación (Google, Facebook).
- Mejora en el sistema de búsqueda de contactos.
- Arreglado un problema con la sincronización de mensajes en dispositivos múltiples.
- Reducción del tiempo de carga de las imágenes enviadas por el chat.

## Version 1.5.0
- Añadida funcionalidad de mensajes de voz en el chat.
- Mejora en la seguridad del sistema de autenticación.
- Corrección de errores relacionados con la carga de archivos grandes.
- Actualización de las librerías utilizadas para mejorar compatibilidad y rendimiento.

## Version 1.6.0
- Incorporación de un tema oscuro para la interfaz de usuario.
- Solucionado un problema que causaba el cierre inesperado de la aplicación en algunos dispositivos Android.
- Mejoras en la accesibilidad de la interfaz.
- Añadida la opción de configurar respuestas automáticas en el chat.

## Version 1.7.0
- Implementación de un sistema de estadísticas para el usuario (mensajes enviados, archivos compartidos, etc.).
- Corrección de problemas menores reportados por los usuarios.
- Mejora en la interfaz del administrador.
- Actualización del protocolo de comunicación para mejorar la estabilidad en conexiones lentas.

## Version 1.8.0
- Añadida la funcionalidad de videollamadas en alta definición.
- Solucionado un problema que causaba latencia en las notificaciones push.
- Mejoras en la integración con servicios de almacenamiento en la nube.
- Ajustes en la política de privacidad para cumplir con las normativas internacionales.

## Version 1.9.0
- Implementación de un sistema de moderación para chats grupales.
- Corrección de un problema con los emojis que no se mostraban correctamente en algunos dispositivos.
- Mejora en el soporte para idiomas asiáticos.
- Optimización general del rendimiento de la aplicación.

## Version 2.0.0
- Rediseño completo de la interfaz de usuario para mejorar la experiencia del usuario.
- Implementación de cifrado de extremo a extremo para todos los mensajes.
- Incorporación de herramientas avanzadas para la gestión de grupos.
- Migración a una nueva arquitectura para mejorar la escalabilidad.
"""

query = input("Ingrese su consulta: ")
answer = main_agent(changelog, query)
print(answer)


Version 1.6.0:
    
Características:
- Incorporación de un tema oscuro para la interfaz de usuario.
- Solucionado un problema que causaba el cierre inesperado de la aplicación en algunos dispositivos Android.
- Añadida la opción de configurar respuestas automáticas en el chat.

Correcciones de errores:
- Sin correcciones

Optimizaciones:
- Mejoras en la accesibilidad de la interfaz.


Versiones cercanas:
- 


